# The effect of normalising the input of the PINN

> In this notebook, I checked the eefect of normalising the input by usiing lower and upper bounds (lb, ub) on the outputs and the first and the second order derivatives.

### Verdict:

> The outputs remains the same, even if we change the scale of the data, say L. However, the first and the second order derivatives are scaled as $L^{-1}$ and $L^{-2}$ respectively.

In [22]:
%config Completer.use_jedi = False
import sys
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import numpy as np


sys.path.append("/end/home/rh2310/morpho_repo/turing_codebase")
from turing.utils import *
import turing.pinns as tu
from turing.loss_functions import *

In [17]:
tf.random.set_seed(43)
np.random.seed(43)
tf.compat.v1.set_random_seed(43)

# Create a PINN for L=1.0 by tunning the ub

In [18]:
layers = [3, 64, 64, 64, 64, 2]
pinn = tu.NN(layers, lb=np.array([0.0, 0.0, 0.0]), ub=np.array([1.0, 1.0, 1.0]), dtype=tf.float64)

# Check the output and the gradients for (0, 0, 0) and (1, 1, 1). 

In [19]:
inputs = tf.constant(np.array([[0.0, 0.0, 0.0], [1.0, 1.0, 1.0]]), dtype=tf.float64)

with tf.GradientTape(persistent=True) as tape:
    tape.watch(inputs)
    with tf.GradientTape(persistent=True) as tape2:
        tape2.watch(inputs)
        outputs = pinn(inputs)
    g1 = tape2.gradient(outputs, inputs)
    
print(outputs)

g2 = tape.gradient(g1, inputs)
print(g1)
print(g2)

tf.Tensor(
[[-0.01898138 -0.03388576]
 [ 0.01898138  0.03388576]], shape=(2, 2), dtype=float64)
tf.Tensor(
[[ 0.01341942 -0.02162235  0.10655636]
 [ 0.01341942 -0.02162235  0.10655636]], shape=(2, 3), dtype=float64)
tf.Tensor(
[[-0.01161232 -0.06457729  0.10123398]
 [ 0.01161232  0.06457729 -0.10123398]], shape=(2, 3), dtype=float64)


# Create another PINN with the same weights, but this tme the dimension is L=10.0, or one order of magnitude larger

In [20]:
tf.random.set_seed(43)
np.random.seed(43)
tf.compat.v1.set_random_seed(43)

layers = [3, 64, 64, 64, 64, 2]
pinn2 = tu.NN(layers, lb=np.array([0.0, 0.0, 0.0]), ub=np.array([10.0, 10.0, 10.0]), dtype=tf.float64)

# Then the outputs for (0, 0, 0) and (10, 10, 10) are exactly the same, but the first and the second order derivatives or scaled as $L^{-1}$ and $L^{-2}$, as expected.

In [21]:
inputs = tf.constant(np.array([[0.0, 0.0, 0.0], [10.0, 10.0, 10.0]]), dtype=tf.float64)

with tf.GradientTape(persistent=True) as tape:
    tape.watch(inputs)
    with tf.GradientTape(persistent=True) as tape2:
        tape2.watch(inputs)
        outputs = pinn2(inputs)
    g1 = tape2.gradient(outputs, inputs)
    
print(outputs)

g2 = tape.gradient(g1, inputs)
print(g1)
print(g2)

tf.Tensor(
[[-0.01898138 -0.03388576]
 [ 0.01898138  0.03388576]], shape=(2, 2), dtype=float64)
tf.Tensor(
[[ 0.00134194 -0.00216224  0.01065564]
 [ 0.00134194 -0.00216224  0.01065564]], shape=(2, 3), dtype=float64)
tf.Tensor(
[[-0.00011612 -0.00064577  0.00101234]
 [ 0.00011612  0.00064577 -0.00101234]], shape=(2, 3), dtype=float64)
